In [41]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

from shapely.geometry import box

sys.path.append('../../INFRA_SAP')

import infrasap.market_access as ma  # performs market access
import infrasap.rasterMisc as rMisc  # clips rasters and zonal stats
import infrasap.UrbanRaster as urban # calculates urban populations

In [36]:
#Define input datasets
in_folder = "../Data"
FSA_file = os.path.join(in_folder, "FSAS", "lfsa000a16a_e.shp")
facilities_file = os.path.join(in_folder, "HEALTH_FACILITIES", "fcb06d64-0960-492d-b93c-54cdb4440603202031-1-9da4d.f4ym6p.shp")
travel_surface = os.path.join(in_folder, "TravelTime", "global_friction_surface.tif")
canada_pop = "D:/Work/Canada/can_ppp_2020_1km_Aggregated_UNadj.tif"

inR = rasterio.open(travel_surface)
inD = gpd.read_file(FSA_file)
inD = inD.to_crs(inR.crs)
inF = gpd.read_file(facilities_file)
inF = inF.to_crs(inF.crs)
# Select FSAs that intersect AOI
facilities_extent = box(*inF.total_bounds)
selD = inD.loc[inD['geometry'].apply(lambda x: x.intersects(facilities_extent))]

# Extract local population
inP = rasterio.open(canada_pop)
pop_file = os.path.join(in_folder, "wp_2020.tif")
rMisc.clipRaster(inP, selD, pop_file)

In [47]:
# calculate urban
urban_file = os.path.join(in_folder, "urban.tif")
hd_urban_file = os.path.join(in_folder, "hd_urban.tif")
ub_calc = urban.urbanGriddedPop(pop_file)
urban = ub_calc.calculateUrban(raster = urban_file)
hdurban = ub_calc.calculateUrban(densVal=1500, totalPopThresh=50000, smooth=True, queen=True, raster = hd_urban_file)

In [88]:
# Create FSA centroids
inD_centroids = selD.copy()
inD_centroids['geometry'] = inD_centroids['geometry'].apply(lambda x: x.centroid)
inD_centroids.to_file(facilities_file.replace(".shp", "_centroids.geojson"), driver="GeoJSON")

In [28]:
# Calculate traveltime to health facilities
travelD = inR.read()
travelD = travelD * 1000
mcp = graph.MCP_Geometric(travelD[0,:,:])
out_travel_time = os.path.join(in_folder, "travel_time_longtermcare.tif")

costs, traceback = ma.calculate_travel_time(inR, mcp, inF, out_travel_time)

In [68]:
# create combo population and travel time raster
importlib.reload(rMisc)

out_pop = os.path.join(in_folder, "wp_2020_normalized.tif")
out_pop_tt = os.path.join(in_folder, "wp_2020_tt.tif")

popR = rasterio.open(pop_file)
ttR = rasterio.open(out_travel_time)
rMisc.standardizeInputRasters(popR, ttR, out_pop)
popR = rasterio.open(out_pop)
popD = popR.read()
ttD = ttR.read()
tt_pop = popD * ttD
tt_pop = tt_pop.astype(popR.meta['dtype'])
with rasterio.open(out_pop_tt, 'w', **popR.meta) as out_pop:
    out_pop.write(tt_pop)



In [76]:
importlib.reload(rMisc)
# Calculate centroid vs average traveltime
res = rMisc.zonalStats(selD, out_travel_time, minVal=0)
tt_res = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])

res = rMisc.zonalStats(selD, pop_file, minVal=0)
tt_pop = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])

res = rMisc.zonalStats(selD, out_pop_tt, minVal=0)
tt_pop_tt = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])
tt_pop_tt['SUM'] = tt_pop_tt['SUM'] / tt_pop['SUM']
tt_pop_tt['MEAN'] = tt_pop_tt['MEAN'] / tt_pop['MEAN']

In [81]:
selD['POP'] = tt_pop['SUM']
selD['TT_NORM'] = tt_res['MEAN']
selD['TT_POP'] = tt_pop_tt['MEAN']

C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [93]:
centroid_geoms = [(x.x, x.y) for x in inD_centroids['geometry']]
centroid_res = ttR.sample(centroid_geoms)
selD['tt_CENTROID'] = [x[0] for x in list(centroid_res)]

C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
selD.head()

,CFSAUID,PRUID,PRNAME,geometry,POP,TT_NORM,TT_POP,tt_CENTROID
0,B0S,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-65.01952 45.05329, -65.01952 45.053...",16789.140625,37.045488,9.565978,36.255844
1,B0T,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-64.87545 44.47682, -64.86791 44.470...",14859.835938,55.296662,10.222946,143.320558
2,B0V,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-65.69960 44.60913, -65.69984 44.609...",5707.965820,37.038879,6.706394,34.643701
3,B0W,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-65.63450 44.55529, -65.61838 44.540...",25763.494141,65.585466,12.260307,22.724979
4,B1S,12,Nova Scotia / Nouvelle-Écosse,"POLYGON ((-60.18769 46.13066, -60.18747 46.130...",10150.089844,2.324821,1.163623,0.600000


In [94]:
selD.to_file(os.path.join(in_folder, "FSA_attributed.geojson"), driver="GeoJSON")